### Example 1: a basic pytorch model.

In [1]:
onnx_model = "example1.onnx"

In [2]:
import torch
import torch.nn as nn

In [3]:
class SampleNet(nn.Module):
    def __init__(self):
        super(SampleNet, self).__init__()
        self.m = 10
        self.c = 20
    
    def forward(self, x):
        return self.m * x + self.c

In [4]:
model = SampleNet()
model.eval()

SampleNet()

In [5]:
x = torch.LongTensor([1])
out = model(x)

In [6]:
out

tensor([30])

In [7]:
torch.onnx.export(
model,
x,
onnx_model,
opset_version=11,
do_constant_folding=True,
input_names=['input'],
output_names=['output'])

In [8]:
!pip install netron

     |████████████████████████████████| 1.4 MB 1.6 MB/s eta 0:00:01


In [9]:
import netron
netron.start(onnx_model, port=8080)

Serving 'example1.onnx' at http://localhost:8080


### use the onnx model

In [10]:
import onnxruntime as ort
import numpy as np
sess = ort.InferenceSession(onnx_model)

In [11]:
sess.get_inputs()[0].name

'input'

In [12]:
# check out the signature of sess.run - it has to have the output, then something like a feed_dict.

# either pass the feed dict directly
outs = sess.run(['output'],
                       {
                         'input': np.array([23], dtype=np.int64)  
                       })

print(outs)

[array([250], dtype=int64)]


In [13]:
# or, prepare it first and then pass it in.

inputs = {'input': np.array([4], dtype=np.int64)}

outs = sess.run(['output'],
                       inputs)

print(outs)

[array([60], dtype=int64)]
